In [1]:
import os
from pathlib import Path

from es_sfgtools.processing.assets.siteconfig import SiteConfig
from es_sfgtools.processing.operations.site_ops import (
    CTDfile_to_svp,
)
from es_sfgtools.processing.pipeline.data_handler import DataHandler
from es_sfgtools.utils.loggers import BaseLogger

pride_path = Path.home() / ".PRIDE_PPPAR_BIN"
os.environ["PATH"] += os.pathsep + str(pride_path)

In [2]:
main_dir = Path("/Users/franklyndunbar/Project/SeaFloorGeodesy/Data/SFGMain")
BaseLogger.set_format_minimal()
dh = DataHandler(main_dir)

In [3]:
network = "cascadia-gorda"
station = "NCC1"
survey = "2024_A_1126"
dh.change_working_station(network=network, station=station, survey=survey)

In [4]:
sv3_pipeline,config = dh.get_pipeline_sv3()
config.novatel_config.override = False
config.dfop00_config.override = True
sv3_pipeline.config = config
sv3_pipeline.run_pipeline()

Processing Novatel 770 data for cascadia-gorda NCC1 2024_A_1126
Getting assets for cascadia-gorda NCC1 2024_A_1126 AssetType.NOVATEL770
Processing Novatel 000 data for cascadia-gorda NCC1 2024_A_1126
Getting assets for cascadia-gorda NCC1 2024_A_1126 AssetType.NOVATEL000
Getting assets for cascadia-gorda NCC1 2024_A_1126 AssetType.RINEX
Getting assets for cascadia-gorda NCC1 2024_A_1126 AssetType.KIN


/Users/franklyndunbar/Project/SeaFloorGeodesy/es_sfgtools/src/es_sfgtools/processing/operations/gnss_ops.py:721: UserWarning: No kin file generated from RINEX /Users/franklyndunbar/Project/SeaFloorGeodesy/Data/SFGMain/cascadia-gorda/NCC1/2024_A_1126/intermediate/NCC12760.24o
  warn(response)
/Users/franklyndunbar/Project/SeaFloorGeodesy/es_sfgtools/src/es_sfgtools/processing/operations/gnss_ops.py:721: UserWarning: No kin file generated from RINEX /Users/franklyndunbar/Project/SeaFloorGeodesy/Data/SFGMain/cascadia-gorda/NCC1/2024_A_1126/intermediate/NCC12820.24o
  warn(response)
/Users/franklyndunbar/Project/SeaFloorGeodesy/es_sfgtools/src/es_sfgtools/processing/operations/gnss_ops.py:721: UserWarning: No kin file generated from RINEX /Users/franklyndunbar/Project/SeaFloorGeodesy/Data/SFGMain/cascadia-gorda/NCC1/2024_A_1126/intermediate/NCC12690.24o
  warn(response)
/Users/franklyndunbar/Project/SeaFloorGeodesy/es_sfgtools/src/es_sfgtools/processing/operations/gnss_ops.py:721: UserWarn

Processing Rinex Files:   0%|          | 0/9 [00:00<?, ?it/s]

Getting assets for cascadia-gorda NCC1 2024_A_1126 AssetType.KIN
Getting assets for cascadia-gorda NCC1 2024_A_1126 AssetType.KINRESIDUALS
Getting assets for cascadia-gorda NCC1 2024_A_1126 AssetType.DFOP00


Processing DFOP00 Files:   0%|          | 0/25 [00:00<?, ?it/s]

Updating shotdata with interpolated gnss data


In [5]:
ncc1_2024_config = dh.station_dir / "NCC1_2024_config.yaml"
svp_path = dh.station_dir / "NCC1_CTD_2021_fit"
svp_path_processed = dh.station_dir / "svp.csv"
site_path = Path(
    "/Users/franklyndunbar/Project/SeaFloorGeodesy/es_sfgtools/dev/NCC1_Dec3_move_around.json"
)
if not svp_path_processed.exists():
    svp_df = CTDfile_to_svp(svp_path)
    svp_df.to_csv(svp_path_processed)
config = SiteConfig.from_config(ncc1_2024_config)
config.sound_speed_data = svp_path_processed
gp_handler_ncc1= dh.get_garpos_handler(site_config=config)

In [7]:
gp_handler_ncc1.load_campaign_data(site_path)
gp_handler_ncc1.set_campaign("2024_A_1126")

gp_handler_ncc1.prep_shotdata(True)
update_dict = {"rejectcriteria": 2.5}
gp_handler_ncc1.set_inversion_params(update_dict)
# gp_handler_ncc1.run_garpos()

In [ ]:
gp_handler_ncc1.run_garpos()